### **Imports**

In [1]:
from __future__ import annotations

from pathlib import Path

import pandas as pd

from kub.course.plotlib.simulationPlotFactory import SimulationPlotFactory
from kub.course.plotlib.weatherPlotFactory import WeatherPlotFactory
from kub.course.simlib.simulation import FMUSimulation

# Find repository root: check if database/ exists at current location
repo_root = (
    Path.cwd() if (Path.cwd() / "database").exists() else Path.cwd().parent.parent
)
databasePath = repo_root / "database" / "day2"

# Thermal Building Optimization Exercise II: **Integrating Glazing**

This second exercise builds upon the parameters and constraints established in Exercise I (geometry, opaque material thickness, $\rho$, $\lambda$, and layer placement). Using the model **`App4WallsGlazed1Floor1Roof`**, your primary objective remains to **minimize `energyConsumptionTotal`** while maintaining thermal comfort, but now by optimizing the **windows** on each of the four walls.

***

## Learning Objectives

Upon completion of this exercise, you should be able to:

1.  **Analyze Glazing Impact:** Quantify the trade-off between heat losses (conduction) and solar gains (radiation) through transparent elements.

2.  **Optimize Glazing Distribution:** Determine the optimal Window-to-Wall Ratio (WWR) distribution across façades based on building orientation and climate.

3.  **Integrate Design Decisions:** Assess how glazing properties (`UWindow`) and area interact with the building's thermal inertia (opaque envelope from Exercise I).

***

## 1. New Building Elements and Constraints

The building now includes a single window on each wall (`window_1_floor_1` to `window_4_floor_1`).

### Glazing Constraints

* **Window Area Limit:** The area of each window must be strictly **smaller** than the total area of the wall it is attached to.
    * Wall 1 (South): $widthWindow1 \times heightWindow1 < LengthWall1 \times height$
    * Wall 2 (West): $widthWindow2 \times heightWindow2 < LengthWall2 \times height$
    * Wall 3 (North): $widthWindow3 \times heightWindow3 < LengthWall1 \times height$
    * Wall 4 (East): $widthWindow4 \times heightWindow4 < LengthWall2 \times height$
* **Glazing U-value:** You can vary the window's overall heat transfer coefficient (`UWindow`), representing the quality of the glazing.

### Geometric and Opaque Material Constraints

* **All constraints from Exercise I remain active** (Area=$100\,\text{m}^2$, $2.7 \le height \le 3.0\,\text{m}$, $\text{Thickness} \le 0.40\,\text{m}$).
* For this exercise, use the **optimized opaque envelope design** (wall materials and orientation) determined in Exercise I as your starting point.

### New Variables Available for Manipulation

* `widthWindow1`, `heightWindow1` (and 2, 3, 4)
* `UWindow` (Overall Heat Transfer Coefficient, $W/(m^2 \cdot K)$)

***

## 2. Assignment: Glazing Optimization Strategy

Your task is to find the optimal combination of **window size, U-value, and orientation** that complements your optimized opaque envelope to achieve the lowest `energyConsumptionTotal`.

### Suggested Investigation Steps:

1.  **Baseline Glazing Scenario:**

    * Start with a moderate Window-to-Wall Ratio (WWR) (e.g., $10-15\%$ of the total façade area) and an initial `UWindow` (e.g., $2.5 \, \text{W/}(\text{m}^2 \cdot \text{K})$).

    * Ensure the WWR is initially distributed equally for all four façades.

2.  **U-Value Impact Study:**

    * **Vary `UWindow`:** Test a low U-value (e.g., $0.8 \, \text{W/}(\text{m}^2 \cdot \text{K})$, high-performance triple glazing) and a high U-value (e.g., $4.0 \, \text{W/}(\text{m}^2 \cdot \text{K})$, single glazing).

    * **Analysis:** Observe the change in `energyConsumptionTotal` (Graph 3) to quantify the energy penalty of poor glazing and the benefit of high-performance glazing.

3.  **Orientation and Area Optimization:**

    * Determine the **optimal distribution of glazing area** across the four façades (South, West, North, East) by manipulating the `widthWindowX` and `heightWindowX` variables.

    * **Strategy:** Should the South-facing wall maximize passive solar gain, or should the North wall be minimized to reduce heat loss?

    * **Flux Analysis (Graph 5):** Use the solar flux component for the window surfaces (new surfaces 6 to 9 in the `surfacesToAmbience` array) to correlate window size/orientation with energy gain.

4.  **Interaction with Thermal Inertia:**

    * If you maximize solar gains (large windows), observe the interior air temperature (`building.zone_habitable_1.TAir`, Graph 1).

    * **Analysis:** Does your heavy layer placement (from Exercise I) effectively smooth out the temperature peaks caused by solar heat gain through the windows?

***

### Success Criteria:

* The final configuration must adhere to **all geometric and material constraints** (including the new window area limits).

* The final configuration must **minimize `energyConsumptionTotal`** over the simulation period.


# Exercise resolution: ***TO DO***

### **Weather Display**

In [2]:
# Load Weather Data
df_weather = pd.read_csv(databasePath / "YearlyWeather" / "Strasbourg2023.csv")
df_weather["time"] = pd.to_datetime(df_weather["time"], unit="ms")
df_weather.set_index("time", inplace=True)

weather_factory = WeatherPlotFactory()

weather_factory.plot_yearly_temperature(df_weather)
# weather_factory.plot_yearly_solar_radiation(df_weather)

### **Building thermal simulation**

#### Functions

In [10]:
"""missing Function for computing wall_resistance:

"""

def calculate_wall_resistance(Area, hi, he, material_layers):
    """
    Calculates the equivalent thermal resistance (Req) of a wall for N layers.

    Parameters:
    - Area (float): Surface area of the wall (m^2).
    - hi (float): Internal convective heat transfer coefficient (W/(m^2*K)).
    - he (float): External convective heat transfer coefficient (W/(m^2*K)).
    NOTE: For ground/soil contact walls, 'he' should be set to 0.0.
    - material_layers (list of dict): List of the wall's layers.
    Each dict must contain 'e' (thickness in m) and 'lambda' (conductivity in W/(m*K)).

    Returns:
    - Req (float): The equivalent thermal resistance (K/W).
    """

    # --- 1. Calculate Internal Convective Resistance (R_hi) ---
    # Be careful to handle the case where (hi * Area) might be zero (for R_hi = 0).
    # TODO
    R_hi = 1 / (hi * Area) if (hi * Area) != 0.0 else 0.0

    # --- 2. Calculate Total Conduction Resistance (sum of N layers) ---
    R_total_conduction = 0.0
    # Iterate through the material_layers list
    for layer in material_layers:
        thickness = layer["e"]
        lambda_mat = layer["lambda"]
        # Calculate R_conduction for the current layer and add it to R_total_conduction
        # Be careful to handle the case where (lambda_mat * Area) might be zero (for R_cond = 0).
        # TODO
        if (lambda_mat * Area) != 0:
            R_total_conduction += thickness / (lambda_mat * Area)
        else:
            R_total_conduction += 0.0

    # --- 3. Calculate External Convective Resistance (R_he) ---
    # Be careful to handle the case where (he * Area) might be zero (for R_he = 0).
    # TODO
    R_he = 1 / (he * Area) if (he * Area) != 0.0 else 0.0

    # --- 4. Calculate Total Equivalent Resistance (R_eq) ---
    # TODO
    R_eq = R_hi + R_total_conduction + R_he

    return R_eq




def calculate_total_heat_flow_balance(
    wall_balance_data, T_int, T_ext, airChange, L1, L2, H
):
    """
    Calculates the heat flow (Q_flow) for each wall and the total flow, including ventilation.

    Parameters:
    - wall_balance_data (dict): Dictionary containing the necessary data for the balance.
      Each key is the wall name, and the value is a dict containing:
      'R_eq' (Equivalent Resistance) and 'Delta_T' (T_int - T_target_surface).
    - T_int (float): Reference internal temperature (°C).
    - T_ext (float): Reference external temperature for ventilation (°C).
    - airChange (float): Air change rate (vol/s).
    - L1, L2, H (float): Building dimensions for volume calculation (m).

    Returns:
    - (dict): Individual flows, Q_walls_total, Q_vent, and Q_flow_total.
    """

    # --- Constants for Ventilation ---
    rho_air = 1.2  # Air density (kg/m^3)
    cp_air = 1006.0  # Specific heat capacity of air (J/(kg*K))

    # --- 1. Calculate Heat Loss through Walls (Transmission) ---
    results = {}
    Q_walls_total = 0.0

    for wall_name, data in wall_balance_data.items():
        R_eq = data["R_eq"]
        Delta_T = data["Delta_T"]

        # Calculate Heat Flow (Q_flow = Delta_T / R_eq)
        # Note: Handle the case where R_eq is 0 to avoid division by 0.
        # TODO
        Q_flow = Delta_T / R_eq if R_eq != 0.0 else 0.0

        # Store result for this specific wall
        results[f"Q_flow_{wall_name}"] = Q_flow

        # Add the current wall's Q_flow to the total wall heat loss
        # TODO
        Q_walls_total += Q_flow

    # --- 2. Calculate Heat Loss through Ventilation ---

    # Calculate the building volume based on dimensions (L1, L2, H)
    # TODO
    Volume = L1 * L2 * H

    # --- Pre-calculated flow rates (Physics) ---
    V_dot_air = airChange * Volume  # Volumetric flow rate (m^3/s)
    m_dot_air = rho_air * V_dot_air  # Mass flow rate (kg/s)
    Delta_T_vent = T_int - T_ext  # Ventilation is always T_int - T_ext

    # Calculate Q_vent using the formula: Q = m_dot * cp * Delta_T
    # TODO
    Q_vent = 0.34 * V_dot_air * Delta_T_vent

    # --- 3. Final Result ---

    # Calculate the total heat flow (Sum of walls + ventilation)
    # TODO
    Q_flow_total = Q_walls_total + Q_vent

    results["Q_walls_total"] = Q_walls_total
    results["Q_vent"] = Q_vent
    results["Q_flow_total"] = Q_flow_total

    return results["Q_flow_total"]


In [3]:
# input variables
inputs2 = {
    # ---------- WINDOW GEOMETRY AND U-VALUE
    # dimensions and overall heat transfer coefficient (U-value) for the windows
    "widthWindow1": 1,
    "widthWindow2": 1,
    "widthWindow3": 1,
    "widthWindow4": 1,
    "heightWindow1": 1,
    "heightWindow2": 1,
    "heightWindow3": 1,
    "heightWindow4": 1,
    "UWindow": 1,
    # ---------- WALL PROPERTIES (INTERIOR AND EXTERIOR LAYERS)
    # (thickness, thermal conductivity, density, specific heat)
    "eIntWall": 1,
    "eExtWall": 1,
    "lambdaIntWall": 1,
    "lambdaExtWall": 1,
    "rhoIntWall": 1,
    "rhoExtWall": 1,
    "cIntWall": 1,
    "cExtWall": 1,
    # ---------- ROOF PROPERTIES (INTERIOR AND EXTERIOR LAYERS)
    # (thickness, thermal conductivity, density, specific heat)
    "eIntRoof": 1,
    "eExtRoof": 1,
    "lambdaIntRoof": 1,
    "lambdaExtRoof": 1,
    "rhoIntRoof": 1,
    "rhoExtRoof": 1,
    "cIntRoof": 1,
    "cExtRoof": 1,
    # ---------- BOTTOM/GROUND PROPERTIES (INTERIOR AND EXTERIOR LAYERS)
    # (thickness, thermal conductivity, density, specific heat)
    "eIntBottom": 1,
    "eExtBottom": 1,
    "lambdaIntBottom": 1,
    "lambdaExtBottom": 1,
    "rhoIntBottom": 1,
    "rhoExtBottom": 1,
    "cIntBottom": 1,
    "cExtBottom": 1,
    # ---------- GEOMETRY, ORIENTATION, AND AIR CHANGE
    # (dimensions of the thermal zone and wall orientation)
    "LengthWall1": 10,
    "LengthWall2": 10,
    "height": 3,
    # ---------- SURFACE ABSORPTIVITY
    "absWall1": 1,
    "absWall2": 1,
    "absWall3": 1,
    "absWall4": 1,
    "absRoof": 1,
    # ---------- HEATING AND COOLING POWER + AIRCHANGE
    "heatingPower": 10000,
    "coolingPower": 1,
    "airChange": 0.5,
}

In [4]:
# variables to display
temperatureList2 = ["ambience.TAirRef", "building.zone_habitable_1.TAir"]
energyList2 = [
    "energyConsumptionHeating",
    "energyConsumptionCooling",
    "energyConsumptionTotal",
]
convectiveCoeffList2 = ["Window1_he", "Window2_he", "Window3_he", "Window4_he"]

In [5]:
# simulation
sim2 = FMUSimulation(
    databasePath
    / "BuildingModels"
    / "Exercices_Optimization_LoD0Bui4WallsGlazed1Floor1Roof_App4WallsGlazed1Floor1Roof.fmu"
)

sim2.initialize(startTime=0.0, stopTime=86400 * 365, timeStep=3600.0)
sim2.initParameters(inputs2)
sim2.exitInitialization()

data2 = sim2.run(temperatureList2 + convectiveCoeffList2 + energyList2)

sim2Factory = SimulationPlotFactory()

In [8]:
# Plot: evolution of temperature
sim2Factory.plot_multi_curves(
    time=data2["time"],
    data_type="temperature",
    data_dict={
        "Exterior Temperature (°C)": data2["ambience.TAirRef"],
        "Interior Temperature (°C)": data2["building.zone_habitable_1.TAir"],
    },
    title="Graph 1: Evolution of Temperature",
);

In [9]:
# Plot: convective coefficients
sim2Factory.plot_multi_curves(
    time=data2["time"],
    data_type="raw",
    data_dict={
        "Window1_he": data2["Window1_he"],
        "Window2_he": data2["Window2_he"],
        "Window3_he": data2["Window3_he"],
        "Window4_he": data2["Window4_he"],
    },
    title="Graph 3: Convective Coefficients",
);